# Исследование надежности заемщиков.

Необходимо разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Результаты исследования будут учтены при построении модели кредитного скоринга — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

**Цель исследования** 

Проверим следующие гипотезы:

  1. Есть ли зависимость между количеством детей и возвратом кредита в срок?
  2. Есть ли зависимость между семейным положением и возвратом кредита в срок?
  3. Есть ли зависимость между уровнем дохода и возвратом кредита в срок?
  4. Как разные цели кредита влияют на его возврат в срок?

**Ход исследования**

Исследование пройдёт в три этапа:

1. Обзор данных.
2. Предобработка данных.
3.  Проверка гипотез.


## Обзор данных

In [596]:
import pandas as pd
import numpy as np

In [597]:
bank = pd.read_csv('/datasets/data.csv')
bank.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [598]:
bank.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Входные данные от банка — статистика о платёжеспособности клиентов.

Итак, в таблице двенадцать столбцов. Тип данных в столбцах — object, int, float.

Согласно документации к данным:

- `children` — количество детей в семье
- `days_employed` — общий трудовой стаж в днях
- `dob_years` — возраст клиента в годах
- `education` — уровень образования клиента
- `education_id` — идентификатор уровня образования
- `family_status` — семейное положение
- `family_status_id` — идентификатор семейного положения
- `gender` — пол клиента
- `income_type` — тип занятости
- `debt` — имел ли задолженность по возврату кредитов
- `total_income` — ежемесячный доход
- `purpose` — цель получения кредита

В названиях колонок нарушений стиля нет. Количество значений в столбцах различается. 

Значит, в данных есть пропущенные значения.


**Выводы**

В каждой строке таблицы описаны потенциальные заемщики. При этом, часть данных содержит информацию о самом заемщике, а другая - его доходы и кредитную историю.

Предварительно можно утверждать, что данных достаточно для проверки гипотез. 
Поскольку в данных есть пропуски, займемся устранением данной проблемы в первую очередь. 

<div class="alert alert-success">
<font size="5"><b>Комментарий ревьюера</b></font>

Успех: Подумай также о возможных причинах появления пропусков, а также о том, являются ли они случайными или нет. Можно высказать предварительные варианты их обработки. 

Первый взгляд на таблицу выполнен.

</div>

## Предобработка данных

### Заполнение пропусков

Посчитаем сколько пропущенных значений в таблице:

In [599]:
bank.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

In [600]:
bank['days_employed'].isna().mean()

0.10099883855981417

Пропущенные значения присутствуют в столбцах с количественными переменными `days_employed`, `total_income`. Их доля составляет 10% от общего количества имеющихся данных, а значит эти пропуски важно заполнить явным значением.

Поскольку первый из столбцов содержит отрицательные значения, пропуски в нем необходимо заполнить медианным значением. 

In [601]:
days_median = bank['days_employed'].median()
bank['days_employed'] = bank['days_employed'].fillna(days_median)

Cтолбец `total_income` содержит данные о доходах. Чтобы адекватно отразить среднюю тенденцию лучше заменить пропуски медианным значением, нивелируя влияние выбросов.

In [602]:
bank['total_income']. min()

20667.26379327158

In [603]:
bank['total_income']. max()

2265604.028722744

In [604]:
income_median = bank['total_income'].median()
income_median

145017.93753253992

In [638]:
# группировка по типу занятости
bank.groupby('income_type')['total_income'].count()

income_type
безработный            2
в декрете              1
госслужащий         1451
компаньон           5047
пенсионер           3812
предприниматель        2
сотрудник          11015
студент                1
Name: total_income, dtype: int64

In [606]:
inc_pivot = bank.pivot_table(index= 'income_type', values = 'total_income', aggfunc = ['count', 'sum', 'median'])
inc_pivot

                       count           sum         median
                total_income  total_income   total_income
income_type                                              
безработный                2  2.626795e+05  131339.751676
в декрете                  1  5.382913e+04   53829.130729
госслужащий             1312  2.242186e+08  150447.935283
компаньон               4577  9.264647e+08  172357.950966
пенсионер               3443  4.721299e+08  118514.486412
предприниматель            1  4.991631e+05  499163.144947
сотрудник              10014  1.616062e+09  142594.396847
студент                    1  9.820163e+04   98201.625314


In [607]:
incomes = incomes_pivot['median'].values
incomes

array([[131339.75167621],
       [ 53829.13072906],
       [150420.15027572],
       [172517.41890716],
       [118480.83740785],
       [499163.14494709],
       [142587.58897641],
       [ 98201.62531401]])

In [608]:
bank['total_income'] = bank['total_income'].fillna(0)

In [639]:
# функция замены нулевых значений на медианы, в зависимостри от типа занятости
def for_null(row):
    person = row['income_type']
    money = row['total_income']
    if money == 0:
        for i in range(len(incomes)):
            if person == inc_pivot.index[i]:
                return incomes[i][0]

In [610]:
bank['total_income_new'] = bank.apply(for_null, axis = 1)
bank['total_income_new'] = bank['total_income_new'].fillna(0)

In [611]:
bank['total_income'] = bank['total_income_new'] + bank['total_income']

150420.15027571545


In [612]:
# проверка пропусков в данных
bank.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
total_income_new    0
dtype: int64

Как показала проверка, пропущенные значения отсутствуют в таблице.

### Проверка данных на аномалии и исправления.

Перед поиском дубликатов проведем проверку данных.

Можно заметить, что в столбце `education` встречаются одинаковые значения, записанные в разном регистре. Вероятная причина их появления - человеческий фактор. 

In [613]:
bank['education'].unique()

array(['высшее', 'среднее', 'Среднее', 'СРЕДНЕЕ', 'ВЫСШЕЕ',
       'неоконченное высшее', 'начальное', 'Высшее',
       'НЕОКОНЧЕННОЕ ВЫСШЕЕ', 'Неоконченное высшее', 'НАЧАЛЬНОЕ',
       'Начальное', 'Ученая степень', 'УЧЕНАЯ СТЕПЕНЬ', 'ученая степень'],
      dtype=object)

In [614]:
bank['education'] = bank['education'].str.lower()

In [615]:
bank['children'].unique()

array([ 1,  0,  3,  2, -1,  4, 20,  5], dtype=int64)

В колонке `children` замечаем несколько ошибок в данных - количество детей не может быть отрицательным; так же 20 является ошибочным значением. Вероятно, ошибки возникли в процессе заполнения таблицы данными или ее выгрузки. 

При таких значениях данные непригодны для анализа и для их замены недостаточно информации. Избавимся от ненужных данных, их количество незначительно:

In [616]:
bank = bank.loc[(bank['children'] != -1) & (bank['children'] != 20)]

In [617]:
bank['days_employed'].unique()

array([-8437.67302776, -4024.80375385, -5623.42261023, ...,
       -2113.3468877 , -3112.4817052 , -1984.50758853])

В столбце `days_employed`, содержащим информацию от трудовом стаже, присутствуют отрицательные значения. Положительные значения соответствуют значениям типа занятости в столбце `income_type` пенсионер. 
Оставляем данный столбец без изменений, проследим его влияние при проверке гипотез.

In [618]:
bank['gender'].value_counts()

F      14154
M       7247
XNA        1
Name: gender, dtype: int64

В столбце `gender` присутствует  одно аномальное значение. Оно, вероятно, появилось тут ошибочно. Заменим его на значение, которое чаще всего встречается в данном столбце - F.

In [619]:
bank['gender'] = bank['gender'].replace("XNA", 'F')

In [620]:
bank['dob_years'].unique()

array([42, 36, 33, 32, 53, 27, 43, 50, 35, 41, 40, 65, 54, 56, 26, 48, 24,
       21, 57, 67, 28, 63, 62, 47, 34, 68, 25, 31, 30, 20, 49, 37, 45, 61,
       64, 44, 52, 46, 23, 38, 39, 51,  0, 59, 29, 60, 55, 58, 71, 22, 73,
       66, 69, 19, 72, 70, 74, 75], dtype=int64)

Нельзя не отметить, что в столбце `dob_years` присутствуют нулевые значения возраста. Доля таких значений незначительная, что не должно существенно повлиять на результаты проверки гипотез. Проверим это в процессе исследования.

**Изменение типов данных**

Для удобства работы со значениями зарплаты изменим тип данных столбца `total_income` на целочисленный.

In [621]:
bank['total_income'] = bank['total_income'].astype('int')

**Удаление дубликатов**

После очистки от аномалий посчитаем явные дубликаты в данных:

In [622]:
# поиск явных дубликатов
bank.duplicated().sum()

71

Доля дубликатов в данных незначительная. 
Удалим их специальным методом с формированием новых индексов в обновленном датафрейме.

In [623]:
bank = bank.drop_duplicates().reset_index(drop = True)

**Выводы**

В процессе предобработки были обнаружены три проблемы в данных:

- пропущенные значения,
- аномальные значения,
- явные дубликаты

Пропущенные значения были заменены на медианные. Это необходимо для более точного анализа данных.
Единственное аномальное значение столбца `gender` заменено на наиболее частое из двух имеющихся типов значений в столбце. Нулевыми значениями столбца `dob_years` мы пренебрегаем, при проверке гипотез проверим какое влияние окажут эти пропуски. Строки с аномальными значениями в столбце `children` удалены.

Дубликаты были удалены, без них исследование станет более точным.

Теперь можно перейти к проверке гипотез.

### Формирование дополнительных датафреймов словарей, декомпозиция исходного датафрейма.

Сформируем отдельные датафреймы для работы над проверкой гипотез.

In [624]:
# новый датафрейм, содержаний информацию об образовании
education = bank.loc[:, ['education', 'education_id']]

In [625]:
# удаление дубликатов в данных
education = education.drop_duplicates().reset_index(drop = True).sort_values(by = 'education_id')
education.head()

,education,education_id
0,высшее,0
1,среднее,1
2,неоконченное высшее,2
3,начальное,3
4,ученая степень,4


In [626]:
family = bank.loc[:, ['family_status', 'family_status_id']]

In [627]:
family = family.drop_duplicates().reset_index(drop = True).sort_values(by = 'family_status_id')
family.head()

,family_status,family_status_id
0,женат / замужем,0
1,гражданский брак,1
2,вдовец / вдова,2
3,в разводе,3
4,Не женат / не замужем,4


Удалим их исходного датафрейма колонки `family_status`, `education`. Используя новые датафреймы мы сможем обращаться к этим данным по соответствующим идентификаторам.

In [628]:
bank.drop(columns =['family_status', 'education'], axis = 1, inplace = True)

### Категоризация дохода.

Как было отмечено ранее, в данных столбца `total_income` наблюдается существенный разброс данных. 
Необходимо сгруппировать их, чтобы данных было достаточно для проверки существующей гипотезы влияния доходности на возврат кредита. Создадим функцию и опишем ею правила классификации доходов:

In [629]:
# функция классификации дохода
def income_group(income):
    if (income >= 0 and income <= 30000):
        return 'E'
    elif (income > 30000 and income <= 50000):
        return 'D'
    elif (income > 50000 and income <= 200000):
        return 'C'
    elif (income > 200000 and income <= 1000000):
        return 'B'
    elif income > 1000000:
        return 'A'
    return 'Уровень дохода неопределен'

In [630]:
# создание нового столбца категоризированных значений
bank['total_income_category'] = bank['total_income'].apply(income_group)
bank.head()

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,total_income_new,total_income_category
0,1,-8437.673028,42,0,0,F,сотрудник,0,253875,покупка жилья,0.0,B
1,1,-4024.803754,36,1,0,F,сотрудник,0,112080,приобретение автомобиля,0.0,C
2,0,-5623.422610,33,1,0,M,сотрудник,0,145885,покупка жилья,0.0,C
3,3,-4124.747207,32,1,0,M,сотрудник,0,267628,дополнительное образование,0.0,B
4,0,340266.072047,53,1,1,F,пенсионер,0,158616,сыграть свадьбу,0.0,C


### Категоризация целей кредита.

Для того, чтобы более точно оценить гипотезы, категоризируем цели получения кредита столбца `purpose`, сгруппируем их по схожим названиям. Для начала определим частоту встречаемости уникальных значений в данном столбце:

In [631]:
# частота встречаемости значений в столбце
bank['purpose'].value_counts()

свадьба                                   790
на проведение свадьбы                     763
сыграть свадьбу                           760
операции с недвижимостью                  672
покупка коммерческой недвижимости         658
покупка жилья для сдачи                   649
операции с жильем                         647
операции с коммерческой недвижимостью     645
жилье                                     641
покупка жилья                             640
покупка жилья для семьи                   637
недвижимость                              631
строительство собственной недвижимости    628
операции со своей недвижимостью           623
строительство жилой недвижимости          620
строительство недвижимости                619
покупка своего жилья                      619
покупка недвижимости                      616
ремонт жилью                              604
покупка жилой недвижимости                602
на покупку своего автомобиля              504
заняться высшим образованием      

In [632]:
# функция классификации целей кредита
def purpose_group(purpose):
    if 'свадьб' in purpose:
        return 'проведение свадьбы'
    elif ('недвиж' in purpose) or ('жил' in purpose):
        return 'операции с недвижимостью'
    elif 'образ' in purpose:
        return 'получение образования'
    elif 'авто' in purpose:
        return 'операции с автомобилем'
    return 'классификация невозможна'

In [633]:
# создание нового столбца
bank['purpose_category'] = bank['purpose'].apply(purpose_group)

### Ответы на вопросы.

Чтобы принять или опровергнуть каждую из гипотез, построим по ним сводные таблицы:

##### Вопрос 1: Существует ли зависимость между количеством детей и возвратом кредита в срок?

In [634]:
# сводная таблица по количеству детей
child_pivot = bank.pivot_table(index = 'children', values='debt', aggfunc =['sum', 'count','mean'])
child_pivot.head(10)

,sum,count,mean
,debt,debt,debt
children,,,
0,1063,14091,0.075438
1,444,4808,0.092346
2,194,2052,0.094542
3,27,330,0.081818
4,4,41,0.097561
5,0,9,0.000000


##### Вывод 1: 

Можно заметить, что в зависимости от количества детей в семье процент невыплаты по уже имеющимся кредитам колеблется в пределах 10%, при этом:
 * потенциальные заемщики, имеющие 4, 1 или 2 детей чаще остальных имеет задолженность по кредиту;
 * заемщики с 3 детьми в 8,18% случаев имеют долг перед банком;
 * согласно данным таблицы наибольшее количество людей не имеют детей, однако среди них самы низкий процент невозврата 7,5%
 
 Значит, существует зависимость между количеством детей и возвратом кредита, однако эта зависимость слабая.

##### Вопрос 2: Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [635]:
# сводная таблица по семейному положению
family_pivot = bank.pivot_table(index = 'family_status_id', values ='debt', aggfunc=['sum', 'count', 'mean'])
family_pivot.head(10)

,sum,count,mean
,debt,debt,debt
family_status_id,,,
0,927,12261,0.075606
1,385,4134,0.093130
2,63,951,0.066246
3,84,1189,0.070648
4,273,2796,0.097639


##### Вывод 2:

Согласно данным таблицы 9,76% клиентов в статусе не замужем/не женат имеют задолженность перед банком. Клиенты с семейным положением женат/замужем составляют наибольшую часть выборки, при этом всего 7,5% из них имеют долг перед банком. Вдовец/вдова имеют всего 6,62% невозврата, что из всей имеющейся статистики наименьший показатель.

Таким образом, выявлена взаимосвязь семейного положения и возврата кредита в положенный срок.

##### Вопрос 3: Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [636]:
# сводная таблица по уровню дохода
income_pivot = bank.pivot_table(index = 'total_income_category', values ='debt', aggfunc = ['sum', 'count', 'mean'])
income_pivot.head(10)

,sum,count,mean
,debt,debt,debt
total_income_category,,,
A,2,25,0.080000
B,354,5014,0.070602
C,1353,15921,0.084982
D,21,349,0.060172
E,2,22,0.090909


##### Вывод 3:

Исходя из имеющихся данных заметим. что:
* чаще всего берут кредиты клиенты с уровнем дохода "C", однако 8,5% из них не возвращают кредит в срок;
* категории с самим низким "E" и самым высоким уровнем дохода "А" в 9% и 8% случаев соответственно имеют задолженность перед банком;
* 6% заемщиков с задолженностью имеют доход "D".

Итак, можно говорить о том, что между уровнем дохода и возвратом кредита существует связь.

##### Вопрос 4: Как разные цели кредита влияют на его возврат в срок?

In [637]:
# сводная таблица по целям кредитования
purpose_pivot = bank.pivot_table(index = 'purpose_category', values ='debt', aggfunc = ['sum', 'count', 'mean'])
purpose_pivot.head(10)

,sum,count,mean
,debt,debt,debt
purpose_category,,,
операции с автомобилем,400,4279,0.093480
операции с недвижимостью,780,10751,0.072551
получение образования,369,3988,0.092528
проведение свадьбы,183,2313,0.079118


##### Вывод 4:

По имеющимся данным можно сделать вывод, что большую часть кредитов составляют "операции с недвижимостью", доля не возврата по ним невысокая и составляет 7,25%. Больший процент задолжености имеют клиенты, которые с целью "получение образования" и "операции с недвижимостью". 

Все это говорит о том, что цели оказывают влияние на образование долга по кредиту.

## Общий вывод:

При построении модели кредитного скоринга — специальной системы оценки способности потенциального заёмщика вернуть кредит банку, мы рассмотрели четыре гипотезы, которые должны дать представление о взаимосвязи того или иного фактора с возвратом кредита в срок.

Для более точного анализа и последующей проверки гипотез была сделана замена пропущенных значений медианным, обработаны аномальные значения.

Чтобы проверить каждую из гипотез были построены соответствующие сводные таблицы.
Итак, при проверке гипотезы о связи количества детей и возврата кредита в срок было установлено, что заемщики без детей имеют низкий процент невозврата кредита. 
При анализе взаимосвязи семейного положения и выплаты кредита установлено, что 9,7% клиентов в статусе не замужем\не женат имеют долг перед банком.
При оценке взаимосвязи доходов и возврата кредита установлено, что клиенты со средним уровнем дохода "С" имеют наибольший из всех показатель невыплаты - 8,5%.
В зависимости от целей кредита было выявлено, что чаще оформляется кредит на недвижимость, но доля невозврата по нему ниже чем у остальных видов кредитов и составляет 7,25%

Таким образом, все имеющиеся гипотезы нашли свое подтверждение.
Однако необходимо комплексное рассмотрение факторов для принятия верного решения для построении модели. Кроме того, необходимо учитывать и платежеспособность клиента, который намерен оформить кредит.